In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from tqdm import tqdm

In [2]:
url = "https://limitlessvgc.com/events/?time=all&type=all&region=all&format=all"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

In [3]:
table = soup.find(class_="table")
thead = table.find("thead")
tbody = table.find("tbody")

In [4]:
def get_row_data(row):
  row = row.findAll("td")
  row[0] = row[0].text
  row[1] = row[1].find("span")["title"]
  event_url = "https://limitlessvgc.com/events" + row[2].find("a")["href"]
  event_id = int(event_url.split('/')[-1])
  row[2] = row[2].text
  row[3] = row[3].text
  row[4] = row[4].text
  row.append(event_id)
  return row

In [5]:
headers = ['date', 'event_country', 'name', 'num_players', 'winner', 'event_id']
rows = [get_row_data(tr) for tr in tbody.findAll("tr")]

In [6]:
tournaments_df = pd.DataFrame(data=rows, columns=headers)

In [7]:
tournaments_df

,date,event_country,name,num_players,winner,event_id
0,12 Oct 24,USA,Regional Louisville,753,Andrew Zheng,364
1,28 Sep 24,Germany,Regional Dortmund,710,Davide Miraglia,389
2,28 Sep 24,Brazil,Regional Joinville,111,Gabriel Agati,378
3,14 Sep 24,USA,Regional Baltimore,647,Nicholas Morales,390
4,17 Aug 24,USA,Worlds Honolulu,694,Luca Ceribelli,358
...,...,...,...,...,...,...
331,19 Jun 10,Spain,Spain Nationals 2010,256,Mario Diaz de Cerio Beltran,244
332,12 Jun 10,France,France Nationals 2010,256,f roggy,243
333,05 Jun 10,Germany,German Nationals 2010,256,Robert Wein,242
334,01 Jun 10,Japan,Japan Nationals 2010,32,Ryo Tajiri,240


# Construindo o DataFrame com a classificação de cada evento

In [8]:
# pegando a lista com a url de cada evento
event_id_list = tournaments_df['event_id'].tolist()
event_url_list = [f"https://limitlessvgc.com/events/{id}/" for id in event_id_list]

In [9]:
def get_event_row_data(row):
  row = row.findAll("td")
  row[0] = row[0].text # get the placement
  row[1] = row[1].text # get the player name
  row[2] = row[2].find("span")["title"] # get the plaayer country
  pokemon_team_data = row[3].findAll("span")
  pokemon_team = [pkmn["title"] for pkmn in pokemon_team_data]
  row[3] = pokemon_team
  return row[:-1]

In [10]:
def build_event_df(event_url):
  # parse the html page
  page = requests.get(event_url)
  soup = BeautifulSoup(page.content, "html.parser")

  # get the table body with all the data
  table = soup.find(class_="table")
  tbody = table.find("tbody")

  # get the headers for our event df
  headers = ['event_id', 'placement', 'player', 'country', 'team']
  event_id = int(event_url.split('/')[-2])
  rows = [[event_id] + get_event_row_data(row) for row in tbody.findAll("tr")]
  df = pd.DataFrame(data=rows, columns=headers)
  return df

In [11]:
def build_pokemon_stats_df(event_url_list):
  df = build_event_df(event_url_list[0])
  for event_url in tqdm(event_url_list[1:]):
    df = pd.concat([df, build_event_df(event_url)], ignore_index=True)
  return df

In [12]:
pokemon_stats_df = build_pokemon_stats_df(event_url_list)

100%|██████████| 335/335 [03:38<00:00,  1.53it/s]


Now we explode the team column, reset the indexes and rename the column to pokemon

In [13]:
pokemon_stats_df = pokemon_stats_df.explode('team') # explode the team column
pokemon_stats_df.reset_index(drop=True, inplace=True) # reset the indexes
pokemon_stats_df.rename(columns={"team": "pokemon"}, inplace=True) # rename the column from team to pokemon
pokemon_stats_df = pokemon_stats_df.loc[pokemon_stats_df['pokemon'] != ""] # drop all rows with empty pokemon
pokemon_stats_df

,event_id,placement,player,country,pokemon
0,364,1,Andrew Zheng,USA,Amoonguss
1,364,1,Andrew Zheng,USA,Incineroar
2,364,1,Andrew Zheng,USA,Sneasler
3,364,1,Andrew Zheng,USA,Garchomp
4,364,1,Andrew Zheng,USA,Porygon2
...,...,...,...,...,...
63211,241,4,Lee Provost,Great Britain,Kyogre
63212,241,4,Lee Provost,Great Britain,Dialga
63213,241,4,Lee Provost,Great Britain,Abomasnow
63214,241,4,Lee Provost,Great Britain,Palkia


Get the ruleset/generation of each event

In [14]:
def get_ruleset_from_event_id(event_id):
  event_url = f"https://limitlessvgc.com/events/{event_id}/"
  page = requests.get(event_url)
  soup = BeautifulSoup(page.content, "html.parser")

  infobox = soup.find(class_="infobox-text")
  format = infobox.find("a").text
  return format

In [15]:
tournaments_df['format'] = tournaments_df['event_id'].apply(lambda id: get_ruleset_from_event_id(id))
tournaments_df

,date,event_country,name,num_players,winner,event_id,format
0,12 Oct 24,USA,Regional Louisville,753,Andrew Zheng,364,Scarlet & Violet - Regulation H
1,28 Sep 24,Germany,Regional Dortmund,710,Davide Miraglia,389,Scarlet & Violet - Regulation H
2,28 Sep 24,Brazil,Regional Joinville,111,Gabriel Agati,378,Scarlet & Violet - Regulation H
3,14 Sep 24,USA,Regional Baltimore,647,Nicholas Morales,390,Scarlet & Violet - Regulation H
4,17 Aug 24,USA,Worlds Honolulu,694,Luca Ceribelli,358,Scarlet & Violet - Regulation G
...,...,...,...,...,...,...,...
331,19 Jun 10,Spain,Spain Nationals 2010,256,Mario Diaz de Cerio Beltran,244,VGC 2010
332,12 Jun 10,France,France Nationals 2010,256,f roggy,243,VGC 2010
333,05 Jun 10,Germany,German Nationals 2010,256,Robert Wein,242,VGC 2010
334,01 Jun 10,Japan,Japan Nationals 2010,32,Ryo Tajiri,240,VGC 2010


In [16]:
tournaments_df.loc[tournaments_df['format']==""]

,date,event_country,name,num_players,winner,event_id,format
77,17 Jul 21,USA,Players Cup 4 Global Finals,800,Renzo Navarro,350,
78,10 Apr 21,USA,Players Cup 3 Global Finals,800,Jonathan Evans,352,


In [17]:
tournaments_df['format'].unique()

array(['Scarlet & Violet - Regulation H',
       'Scarlet & Violet - Regulation G',
       'Scarlet & Violet - Regulation F',
       'Scarlet & Violet - Regulation E',
       'Scarlet & Violet - Regulation D',
       'Scarlet & Violet - Regulation C',
       'Scarlet & Violet - Regulation B',
       'Scarlet & Violet - Regulation A', 'VGC 2022', '', 'VGC 2020',
       'Ultra Series', 'Moon Series', 'Sun Series', 'VGC 2018',
       'VGC 2017', 'VGC 2016', 'VGC 2015', 'VGC 2014', 'VGC 2013',
       'VGC 2012', 'VGC 2011', 'VGC 2010'], dtype=object)

In [18]:
format_to_gen_dict = {
  "VGC 2010": "gen_4",
  "VGC 2011": "gen_5",
  "VGC 2012": "gen_5",
  "VGC 2013": "gen_5",
  "VGC 2014": "gen_6",
  "VGC 2015": "gen_6",
  "VGC 2016": "gen_6",
  "VGC 2017": "gen_7",
  "VGC 2018": "gen_7",
  "Sun Series": "gen_7",
  "Moon Series": "gen_7",
  "Ultra Series": "gen_7",
  "VGC 2020": "gen_8",
  "VGC 2022": "gen_8",
  "Scarlet & Violet - Regulation A": "gen_9",
  "Scarlet & Violet - Regulation B": "gen_9",
  "Scarlet & Violet - Regulation C": "gen_9",
  "Scarlet & Violet - Regulation D": "gen_9",
  "Scarlet & Violet - Regulation E": "gen_9",
  "Scarlet & Violet - Regulation F": "gen_9",
  "Scarlet & Violet - Regulation G": "gen_9",
  "Scarlet & Violet - Regulation H": "gen_9",
}

In [19]:
tournaments_df.iloc[[75, 76, 77, 78, 79, 80]]

,date,event_country,name,num_players,winner,event_id,format
75,19 Mar 22,USA,Regional Salt Lake City,163,Chongjun Peng,293,VGC 2022
76,12 Mar 22,Australia,Regional Brisbane,71,Henry Rich,276,VGC 2022
77,17 Jul 21,USA,Players Cup 4 Global Finals,800,Renzo Navarro,350,
78,10 Apr 21,USA,Players Cup 3 Global Finals,800,Jonathan Evans,352,
79,05 Dec 20,USA,Players Cup 2 Global Finals,800,Wolfe Glick,351,VGC 2020
80,22 Aug 20,Germany,Players Cup 1,800,Santino Tarquinio,323,VGC 2020


We'll assume 2021 is gen 8

In [20]:
tournaments_df.loc[[77, 78], 'format'] = "VGC 2020"

In [21]:
tournaments_df['format'].unique()

array(['Scarlet & Violet - Regulation H',
       'Scarlet & Violet - Regulation G',
       'Scarlet & Violet - Regulation F',
       'Scarlet & Violet - Regulation E',
       'Scarlet & Violet - Regulation D',
       'Scarlet & Violet - Regulation C',
       'Scarlet & Violet - Regulation B',
       'Scarlet & Violet - Regulation A', 'VGC 2022', 'VGC 2020',
       'Ultra Series', 'Moon Series', 'Sun Series', 'VGC 2018',
       'VGC 2017', 'VGC 2016', 'VGC 2015', 'VGC 2014', 'VGC 2013',
       'VGC 2012', 'VGC 2011', 'VGC 2010'], dtype=object)

In [22]:
tournaments_df['format'] = tournaments_df['format'].apply(lambda x: format_to_gen_dict[x])

In [23]:
tournaments_df

,date,event_country,name,num_players,winner,event_id,format
0,12 Oct 24,USA,Regional Louisville,753,Andrew Zheng,364,gen_9
1,28 Sep 24,Germany,Regional Dortmund,710,Davide Miraglia,389,gen_9
2,28 Sep 24,Brazil,Regional Joinville,111,Gabriel Agati,378,gen_9
3,14 Sep 24,USA,Regional Baltimore,647,Nicholas Morales,390,gen_9
4,17 Aug 24,USA,Worlds Honolulu,694,Luca Ceribelli,358,gen_9
...,...,...,...,...,...,...,...
331,19 Jun 10,Spain,Spain Nationals 2010,256,Mario Diaz de Cerio Beltran,244,gen_4
332,12 Jun 10,France,France Nationals 2010,256,f roggy,243,gen_4
333,05 Jun 10,Germany,German Nationals 2010,256,Robert Wein,242,gen_4
334,01 Jun 10,Japan,Japan Nationals 2010,32,Ryo Tajiri,240,gen_4


Finally, we'll need a dataframe of all existing pokemon, to do some analysis on their typings later.

In [24]:
url = "https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_name"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

In [25]:
pokemon_data = []
tables = soup.findAll("table")
for table in tqdm(tables):
  try:
    rows = table.findAll("tr")[1:]
    for row in rows:
      entries = row.findAll("td")
      pokemon_id = int(entries[0].text.replace("#", ""))
      pokemon_name = entries[2].text
      pokemon_type_1 = entries[3].text.replace("\n", "")
      if len(entries) > 4:
        pokemon_type_2 = entries[4].text.replace("\n", "")
      else: pokemon_type_2 = None
      pokemon_data.append([pokemon_id, pokemon_name, pokemon_type_1, pokemon_type_2])
  except:
    continue

100%|██████████| 30/30 [00:00<00:00, 899.46it/s]


In [26]:
pokemon_df = pd.DataFrame(data=pokemon_data, columns=["pokemon_id", "name", "type_1", "type_2"])
pokemon_df = pokemon_df.sort_values(by="pokemon_id").reset_index(drop=True)

In [27]:
pokemon_df

,pokemon_id,name,type_1,type_2
0,1,Bulbasaur,Grass,Poison
1,2,Ivysaur,Grass,Poison
2,3,Venusaur,Grass,Poison
3,4,Charmander,Fire,None
4,5,Charmeleon,Fire,None
...,...,...,...,...
1020,1021,Raging Bolt,Electric,Dragon
1021,1022,Iron Boulder,Rock,Psychic
1022,1023,Iron Crown,Steel,Psychic
1023,1024,Terapagos,Normal,None


Now that we have all the dataframes we'll need, we are going to connect them to postgres.

### Tournaments df

In [28]:
tournaments_df = tournaments_df[['event_id', 'date', 'event_country', 'name', 'num_players', 'winner', 'format']]
tournaments_df = tournaments_df.rename(columns={"date": "event_date", "name": "event_name"})
tournaments_df.head(10)

,event_id,event_date,event_country,event_name,num_players,winner,format
0,364,12 Oct 24,USA,Regional Louisville,753,Andrew Zheng,gen_9
1,389,28 Sep 24,Germany,Regional Dortmund,710,Davide Miraglia,gen_9
2,378,28 Sep 24,Brazil,Regional Joinville,111,Gabriel Agati,gen_9
3,390,14 Sep 24,USA,Regional Baltimore,647,Nicholas Morales,gen_9
4,358,17 Aug 24,USA,Worlds Honolulu,694,Luca Ceribelli,gen_9
5,355,08 Jun 24,USA,International New Orleans,921,Patrick Connors,gen_9
6,359,01 Jun 24,Italy,Special Bologna,668,Ruben Gianzini,gen_9
7,334,25 May 24,USA,Regional Los Angeles,618,Montana Mott,gen_9
8,360,18 May 24,Chile,Regional Santiago,153,Hanns Pizarro,gen_9
9,361,11 May 24,Sweden,Regional Stockholm,256,Michael Kelsch,gen_9


### Pokemon Stats df

In [29]:
pokemon_stats_df = pokemon_stats_df.rename(columns={"player": "player_name", "country": "player_country", "pokemon": "pokemon_name"})
pokemon_stats_df = pokemon_stats_df.drop_duplicates(subset=["event_id", "placement", "player_name", "pokemon_name"])
pokemon_stats_df.head(10)

,event_id,placement,player_name,player_country,pokemon_name
0,364,1,Andrew Zheng,USA,Amoonguss
1,364,1,Andrew Zheng,USA,Incineroar
2,364,1,Andrew Zheng,USA,Sneasler
3,364,1,Andrew Zheng,USA,Garchomp
4,364,1,Andrew Zheng,USA,Porygon2
5,364,1,Andrew Zheng,USA,Gholdengo
6,364,2,Joseph Ugarte,USA,Garchomp
7,364,2,Joseph Ugarte,USA,Jumpluff
8,364,2,Joseph Ugarte,USA,Torkoal
9,364,2,Joseph Ugarte,USA,Dragonite


### Pokemon df

In [30]:
pokemon_df = pokemon_df.rename(columns={"name": "pokemon_name"})
pokemon_df.head(10)

,pokemon_id,pokemon_name,type_1,type_2
0,1,Bulbasaur,Grass,Poison
1,2,Ivysaur,Grass,Poison
2,3,Venusaur,Grass,Poison
3,4,Charmander,Fire,None
4,5,Charmeleon,Fire,None
5,6,Charizard,Fire,Flying
6,7,Squirtle,Water,None
7,8,Wartortle,Water,None
8,9,Blastoise,Water,None
9,10,Caterpie,Bug,None


### Connecting to postgres

In [40]:
from dotenv import load_dotenv
import os

load_dotenv()

# Access variables
PSQL_USER = os.getenv("PSQL_USER")
PSQL_PASSWORD = os.getenv("PSQL_PASSWORD")
HOST = os.getenv("HOST", "localhost")  # Default to 'localhost' if not set
PORT = int(os.getenv("PORT", "5432"))     # Default to '5432' if not set
DB_NAME = os.getenv("DB_NAME")

In [43]:
from sqlalchemy import create_engine

conn = create_engine(f'postgresql://{PSQL_USER}:{PSQL_PASSWORD}@{HOST}:{PORT}/{DB_NAME}')

In [48]:
tournaments_df.to_sql("events", con=conn, if_exists="append", index=False)

336

In [49]:
pokemon_df.to_sql("pokemon", con=conn, if_exists="append", index=False)

25

In [50]:
pokemon_stats_df.to_sql("pokemon_stats", con=conn, if_exists="append", index=False)

397